# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770474832430                   -0.53    9.0         
  2   -2.772137463479       -2.78       -1.28    1.0    164ms
  3   -2.772169842108       -4.49       -2.78    1.0    163ms
  4   -2.772170719294       -6.06       -3.78    2.0    199ms
  5   -2.772170722735       -8.46       -4.24    2.0    185ms
  6   -2.772170723012       -9.56       -5.39    1.0    202ms
  7   -2.772170723015      -11.48       -6.14    2.0    187ms
  8   -2.772170723015      -13.36       -6.46    2.0    196ms
  9   -2.772170723015      -13.86       -7.65    1.0    194ms
 10   -2.772170723015   +  -13.97       -7.96    1.0    182ms
 11   -2.772170723015      -13.88       -9.06    1.0    189ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770568462056                   -0.53    9.0         
  2   -2.7

1.7735580413567154

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770840413627                   -0.53    9.0         
  2   -2.772058064358       -2.91       -1.31    1.0    161ms
  3   -2.772083249177       -4.60       -2.59    1.0    165ms
  4   -2.772083400746       -6.82       -3.58    1.0    197ms
  5   -2.772083416687       -7.80       -4.01    2.0    184ms
  6   -2.772083417788       -8.96       -5.28    1.0    167ms
  7   -2.772083417810      -10.66       -5.56    2.0    222ms
  8   -2.772083417811      -12.16       -6.38    2.0    191ms
  9   -2.772083417811      -14.35       -6.86    1.0    201ms
 10   -2.772083417811   +  -13.83       -7.32    1.0    200ms
 11   -2.772083417811      -13.72       -8.15    1.0    175ms

Polarizability via ForwardDiff:       1.7725349715019227
Polarizability via finite difference: 1.7735580413567154
